In [1]:
import pandas as pd
import sklearn
sklearn.__version__

'1.0.2'

In [2]:
df=pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [ ]:
df.head()